In [2]:
import argparse
import torch
import torchvision
import torch.multiprocessing as mp
import numpy as np
import matplotlib.pyplot as plt
import torch.nn as nn
import torch.nn.functional as F
import torch.distributed as dist
import time
import os
from torchvision.datasets import SVHN
from torchvision.transforms import ToTensor
from torchvision.utils import make_grid
from torch.utils.data.dataloader import DataLoader
from torch.utils.data import random_split
from torch.utils.tensorboard import SummaryWriter

In [5]:
class ConvNet(nn.Module):
    def __init__(self, in_size, out_size):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 6, 5, padding=2)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5, padding=2)
        self.fc1 = nn.Linear(1024, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, out_size)

    def forward(self, x, y=None):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x, start_dim=1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

model = ConvNet(10, 10)

In [10]:
for key, val in model.state_dict().items():
    print(key, val)

 -7.1188e-02],
          [-5.1005e-02,  2.8122e-02,  7.1658e-02,  1.1483e-03,  4.2162e-02],
          [ 7.5244e-02,  6.0441e-02, -5.7477e-03,  4.0756e-02, -5.7998e-04],
          [ 6.6844e-03,  1.7374e-02,  2.1839e-02,  7.5342e-02, -5.5329e-02]],

         [[-4.0788e-03, -1.7668e-02,  6.4110e-02,  5.7433e-02,  2.6472e-02],
          [ 7.3911e-02,  6.0280e-02,  1.8903e-02, -2.0204e-02, -3.9116e-04],
          [ 5.0522e-02,  2.6267e-02,  2.6009e-03, -1.1184e-02, -5.6452e-03],
          [-2.2911e-03,  4.3865e-02,  2.9478e-02, -4.3471e-02,  6.4002e-02],
          [-4.5498e-03, -5.5852e-02, -5.0088e-02, -1.4873e-02,  5.6386e-02]],

         [[ 6.2123e-02,  2.5291e-02, -3.3677e-02, -2.5351e-02,  6.9999e-02],
          [ 7.3327e-02, -3.8054e-02, -5.2676e-02,  5.0923e-02, -5.1793e-03],
          [ 8.0263e-02,  8.0249e-02, -5.8114e-03,  1.4932e-02,  8.9789e-03],
          [-2.0640e-02,  4.3729e-02, -5.1281e-02, -2.4543e-02, -7.8544e-02],
          [-2.1978e-02, -6.3951e-02, -4.0077e-02, -4.7364